In [4]:
# Parameters
input_data_path = ""
batch = ""
label = ""

Jupyter notebook for liger integration algoritm

Author: Erno Hänninen

Title: run_liger.ipynb

Created: 2022-12-05

In [5]:
require(rliger)
require(Seurat)

In [ ]:
sobj = readRDS(input_data_path)
sobj

In [7]:
# This function is from https://github.com/theislab/scib-pipeline/blob/0c7be53b1000864fcd31a7b7594f9a5071204233/scripts/integration/integration.R
runLiger = function(sobj, batch, hvg, k=20, res=0.4, small.clust.thresh=20) {

 # Only counts is converted to liger object. To pass our own normalized data,
  # store it in the "counts" slot
  if (is.null(sobj@assays$RNA)) {
    # Seurat v4
    data <- GetAssayData(sobj, slot = "data")
    SetAssayData(sobj, slot = "counts", new.data = as.matrix(data))
  } else {
    # Seurat v3
    sobj@assays$RNA@counts = sobj@assays$RNA@data
  }

  # Create Liger object
  lobj = seuratToLiger(
    sobj,
    raw.assay="originalexp",
    combined.seurat = T,
    meta.var = batch,
    renormalize = F,
    remove.missing = F
  )

  # We only pass nomarlized data, so store it as such
  lobj@norm.data <- lobj@raw.data

  # Assign hvgs
  lobj@var.genes <- hvg

  lobj <- scaleNotCenter(lobj, remove.missing = F) # Can't do our own scaling atm

  # Use tutorial coarse k suggests of 20.
  lobj <- optimizeALS(lobj, k = k, thresh = 5e-5, nrep = 3)

  lobj <- quantileAlignSNF(lobj, resolution = res, small.clust.thresh = small.clust.thresh)

  # Store embedding in initial Seurat object
  # Code taken from ligerToSeurat() function from LIGER
  inmf.obj <- new(
    Class = "DimReduc", feature.loadings = t(lobj@W),
    cell.embeddings = lobj@H.norm, key = "X_emb"
  )
  sobj@reductions['X_emb'] <- inmf.obj

  return(sobj)
}


In [ ]:

if(file.exists(hvg)) {
	hvg<-unlist(readRDS(hvg), use.names=FALSE)
}else {
	hvg <- rownames(sobj@assays$RNA)
}

liger_sobj = runLiger(sobj, batch, hvg)

saveRDS(liger_sobj, file="../Integrated_adata/liger_output.rds")